### Importações

In [ ]:
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.oauth2 import service_account
import psycopg2

# pip install google-api-python-client google-auth


### Configurações da API do Google Drive

In [ ]:
SERVICE_ACCOUNT_FILE = 'caminho/para/credenciais.json'
SCOPES = ['https://www.googleapis.com/auth/drive.file']

credentials = service_account.Credentials.from_service_account_file(
    SERVICE_ACCOUNT_FILE, scopes=SCOPES)
service = build('drive', 'v3', credentials=credentials)

### Banco de Dados

In [ ]:
# método para subir a imagem no drive
def upload_file_to_drive(file_path, mime_type='image/jpeg'):
    file_metadata = {'name': file_path.split('/')[-1]}
    media = MediaFileUpload(file_path, mimetype=mime_type)
    file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()
    file_id = file.get('id')
    return f'https://drive.google.com/uc?id={file_id}'

# Conectar ao banco de dados PostgreSQL
conn = psycopg2.connect(
    dbname="dbpresencatech2025",
    user="dbpresencatech2025_user",
    password="3xFezRq99dDulmawROUuP5cWTowtAx5K",
    host="dpg-cpqrj6dumphs73b19nag-a.oregon-postgres.render.com",
    port="5432"
)

# Criar um cursor para executar comandos SQL
cur = conn.cursor()

# função para inserir no banco de dados
def insert_candidate_data(nome, genero, nr_inscricao, escola, notebook, jantar, turma, periodo, sala, professor, imagem):
    cur.execute(
        """
        INSERT INTO candidatos (nome, genero, nr_inscricao, escola, notebook, jantar, turma, periodo, sala, professor, imagem)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """,
        (nome, genero, nr_inscricao, escola, notebook, jantar, turma, periodo, sala, professor, imagem)
    )
    conn.commit()
    
# Ler o arquivo Excel
df = pd.read_excel('./Cuidado_com_os_dados.xlsx')

# Processar cada linha do DataFrame
for index, row in df.iterrows():
    # Caminho da imagem
    image_path = row['imagem']
    
    # Fazer o upload da imagem e obter o link
    image_link = upload_file_to_drive(image_path)
    
    # Inserir dados no banco de dados
    insert_candidate_data(row['Nome'], row['Genero'], row['Inscrição'], row['Escola'], row['Notebook'], row['Jantar'], row['Turmas'], row['Dia'], row['Sala'], row['Professor'], image_link)

# Fechar a conexão com o banco de dados
cur.close()
conn.close()